## 1. 효과적인 텍스트 분할 전략
- 모델 성능만큼 중요한 요소라고 생각함.
- LLM은 한 번에 처리할 수 있는 토큰(텍스트의 단위) 수에 제한(컨텍스트 윈도우)이 있음.
- 긴 문서는 검색 및 LLM 처리에 적합하도록 여러 개의 작은 청크(chunk)로 분할해야 함.
- 효과적인 텍스트 분할은 RAG 시스템의 성능에 매우 중요함.

**텍스트 분할의 중요성:**
- **컨텍스트 윈도우 관리**: LLM의 입력 제한을 초과하지 않도록 관리해야 함.
- **검색 정확도 향상**: 관련성 높은 정보만 담긴 작은 청크를 검색하여 LLM에 제공함으로써 답변 품질을 높일 수 있음.
- **비용 효율성**: 필요한 부분만 LLM에 전달하여 API 호출 비용을 절감할 수 있음.

**고려사항:**
- **청크 크기 (`chunk_size`)**: 너무 작으면 문맥이 손실되고, 너무 크면 관련 없는 정보가 포함될 수 있음. 적절한 크기 설정이 중요함.
- **청크 중복 (`chunk_overlap`)**: 청크 간의 연속성을 유지하고, 중요한 정보가 청크 경계에서 잘리는 것을 방지할 수 있음. 다만, 너무 크면 중복 정보가 많아져 비효율적일 수 있음.
- **분할 기준**: 문장, 단락 등 의미론적 단위를 기준으로 분할하는 것이 일반적으로 효과적임.

**💡 팁 및 노하우:**
- **문서 유형별 전략**: 문서의 종류(예: 코드, 논문, 대화, 보고서)에 따라 최적의 분할 전략이 다를 수 있음. 예를 들어, 코드 분할 시에는 함수나 클래스 단위 분할을 고려할 수 있음.
- **메타데이터 활용**: 분할된 청크에 원본 문서명, 페이지 번호, 청크 순서 등의 메타데이터를 포함시키면, RAG 시스템에서 출처 추적이나 디버깅 시 매우 유용함.
- **실험과 검증**: `chunk_size`와 `chunk_overlap` 같은 파라미터는 데이터와 사용 목적에 따라 실험을 통해 최적값을 찾는 과정이 필요함. 분할된 청크의 내용을 직접 확인하여 의도대로 분할되었는지 검증하는 것이 중요함.

### 1-1. RecursiveCharacterTextSplitter

- `RecursiveCharacterTextSplitter`는 가장 일반적으로 사용되는 텍스트 분할기 중 하나임.
- 지정된 구분자(separator) 리스트를 순서대로 적용하여 재귀적으로 텍스트를 분할함.
- 예를 들어, 먼저 문단(`\n\n`)으로 나누고, 각 문단이 너무 길면 문장(`. `)으로, 그래도 길면 단어(` `)로 나누는 식으로 진행됨.

**주요 파라미터:**
- `chunk_size`: 목표 청크 크기 (글자 수 또는 토큰 수).
- `chunk_overlap`: 청크 간 중복되는 글자 수 또는 토큰 수. 문맥 유지를 위해 사용됨.
- `length_function`: 청크 크기를 계산하는 함수 (기본값은 `len`, 즉 글자 수).
- `separators`: 분할에 사용할 구분자 리스트. 우선순위 순서대로 적용됨. (예: `["\n\n", "\n", " ", ""]`)

**장점:**
- 설정이 비교적 간단하고, 일반적인 텍스트에 잘 작동함.
- 재귀적 접근 방식으로 의미론적 경계를 어느 정도 존중하려고 시도함.
- 다양한 구분자를 순차적으로 시도하여 유연하게 분할 가능함.

**단점:**
- `separators` 설정에 따라 성능이 달라질 수 있음. 적절한 구분자 설정이 중요함.
- 고정된 크기를 엄격하게 지키기보다, 구분자를 우선적으로 고려하여 분할하므로 청크 크기가 다소 불균일할 수 있음.
- 복잡한 구조의 문서(예: 표, 다단 문서)에서는 완벽한 의미 단위 분할이 어려울 수 있음.

**💡 팁 및 노하우:**
- `separators` 리스트의 순서가 중요함. 일반적인 텍스트의 경우 `["\n\n" (문단), "\n" (줄바꿈), ". " (문장), " " (단어), "" (글자)]` 순서가 효과적이지만, 마크다운이나 코드 같은 특수 형식은 다른 구분자 우선순위가 필요할 수 있음.
- `chunk_overlap`은 보통 `chunk_size`의 10~20% 정도로 설정하지만, 문서의 특성과 검색 전략에 따라 조절하는 것이 좋음.

In [1]:
from langchain_community.document_loaders import PyPDFLoader

pdf_loader = PyPDFLoader('../data/transformer.pdf')
pdf_docs = pdf_loader.load() # Document 객체의 리스트로 반환

In [2]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter_recursive = RecursiveCharacterTextSplitter(
    chunk_size=1000,        # 각 청크의 최대 글자 수
    chunk_overlap=200,      # 청크 간 중복되는 글자 수 (연속성 유지)
    length_function=len,    # 글자 수를 기준으로 분할 (기본값)
    separators=["\n\n", "\n", ". ", " ", ""],  # 분할 시도 순서: 문단 -> 줄바꿈 -> 마침표 -> 공백 -> 글자
    is_separator_regex=False, # 구분자를 정규표현식으로 해석할지 여부
)

# PDF 문서(pdf_docs)를 분할합니다.
recursive_texts = text_splitter_recursive.split_documents(pdf_docs)
print(f"생성된 텍스트 청크 수: {len(recursive_texts)}")
chunk_lengths = [len(text.page_content) for text in recursive_texts]
print(f"각 청크의 길이 (처음 5개): {chunk_lengths[:5]}")
print(f"각 청크의 길이 (마지막 5개): {chunk_lengths[-5:]}")

생성된 텍스트 청크 수: 52
각 청크의 길이 (처음 5개): [981, 910, 975, 451, 932]
각 청크의 길이 (마지막 5개): [929, 849, 812, 814, 817]


- `RecursiveCharacterTextSplitter`는 이름에서 알 수 있듯이 재귀적으로 텍스트를 분할함.
- 지정된 구분자 리스트를 우선순위에 따라 순차적으로 적용하여, 큰 덩어리에서 시작해 점진적으로 더 작은 단위로 나누어 나감.
- `CharacterTextSplitter`보다 의미론적 경계를 유지하려 노력하며, `chunk_size`를 맞추려고 하지만 구분자를 우선시함.

In [3]:
# 각 청크의 시작 부분과 끝 부분 확인 (중복(overlap)이 어떻게 적용되는지 관찰)
for i, text_chunk in enumerate(recursive_texts[:3]): # 처음 3개 청크만 확인
    print(f"--- 청크 {i+1} (길이: {len(text_chunk.page_content)}) ---")
    print("[시작 부분]")
    print(text_chunk.page_content[:200])
    print("\n[...중략...]\n")
    print("[끝 부분]")
    print(text_chunk.page_content[-200:])
    print("=" * 100 + "\n")

--- 청크 1 (길이: 981) ---
[시작 부분]
Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works.
Attention Is All You Need


[...중략...]

[끝 부분]
the encoder and decoder through an attention
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions

--- 청크 2 (길이: 910) ---
[시작 부분]
mechanism. We propose a new simple network architecture, the Transformer,
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experiments on two machine transla

[...중략...]

[끝 부분]
the
best models from the literature. We show that the Transformer generalizes well to
other tasks by applying it successfully to English constituency parsing both with
large and limited training data.

--- 청크 3 (길이: 975) ---
[시작 부분]
best models from the literature. We show that the Transformer

### 1-2. 정규표현식 사용 (CharacterTextSplitter)

- `CharacterTextSplitter`는 `RecursiveCharacterTextSplitter`의 기반이 되는 분할기임.
- `separator`를 단일 문자열로 받으며, `is_separator_regex=True`로 설정하면 이 `separator`를 정규표현식으로 해석하여 분할함.
- 특정 패턴(예: 문장 종결 부호, 특정 형식의 구분자)을 기준으로 텍스트를 정교하게 나눌 때 유용함.

**주요 파라미터:**
- `separator`: 분할 기준이 되는 문자열 또는 정규표현식.
- `is_separator_regex`: `separator`를 정규표현식으로 다룰지 여부 (기본값 `False`).
- `keep_separator`: 분할 후 각 청크에 구분자를 유지할지 여부 (기본값 `False`).

**정규표현식 예시: `(?<=[.!?])\s+`**
- `(?<=[.!?])`: 긍정형 후방탐색(positive lookbehind)으로, 마침표(.), 느낌표(!), 물음표(?) 중 하나가 앞에 오는 위치를 찾음. 이 문자 자체는 분할 기준에 포함되지 않음 (즉, 잘려나가지 않음).
- `\s+`: 하나 이상의 공백 문자와 일치함.
- 즉, 문장의 끝을 나타내는 구두점(`.`, `!`, `?`) 뒤에 오는 하나 이상의 공백을 기준으로 분할함 (구두점은 유지됨).

**장점:**
- 정규표현식을 통해 매우 유연하고 정교한 분할 규칙을 정의할 수 있음.
- 특정 패턴 기반 분할이 필요할 때 강력한 도구임.

**단점:**
- 정규표현식 작성 및 디버깅이 어려울 수 있음.
- 복잡한 정규표현식은 성능에 영향을 줄 수 있음.
- `chunk_size`를 엄격히 지키기보다는 정규표현식에 의한 분할을 우선함. 따라서, 분할된 조각들을 `chunk_size`에 맞춰 합치는 방식으로 동작하여, 실제 청크 크기가 매우 불균일하게 나올 수 있으며, `chunk_size`는 사실상 분할된 부분들을 합칠 때의 최대 크기 제한처럼 동작함.

**💡 팁 및 노하우:**
- **정규표현식 테스트**: 복잡한 정규표현식은 `regex101.com` 같은 온라인 도구에서 충분히 테스트한 후 사용하는 것이 좋음.
- **`keep_separator` 활용**: 
  - `True`로 설정하면 분할 기준이 된 문자열(정규표현식에 매칭된 부분)을 각 청크의 *시작 부분*에 유지함. 문장 분리 시 마침표 등을 유지하고 싶을 때 유용할 수 있으나, 의도와 다르게 동작할 수 있으므로 주의해야 함. (Langchain의 `CharacterTextSplitter`는 `keep_separator=True`일 때, 구분자를 다음 청크의 시작에 붙이는 경향이 있음. 확인 필요)
  - `False`로 설정하면 구분자는 제거됨. 구분자 자체가 불필요하거나, 후처리 과정에서 별도로 처리할 때 유용함. 예를 들어, 문장 분리 후 앞뒤 공백을 완전히 제거하고 싶을 때 활용 가능.
  - **(코드 예시의 `keep_separator=True` 관련)**: 예시의 정규식 `(?<=[.!?])\s+`에서 `\s+`가 구분자임. `keep_separator=True`이면 이 공백이 다음 청크 시작에 붙을 수 있음. 만약 문장 종결 부호(`.!?`)만 남기고 공백을 제거하고 싶다면, `keep_separator=False`를 사용하거나, 정규식 자체에서 공백을 소비하도록 하고 `keep_separator=False`를 사용하는 것이 더 명확할 수 있음. 또는 분할 후 각 청크에 대해 `.strip()`을 적용하는 것도 방법임.

In [4]:
from langchain_community.document_loaders import JSONLoader

# JSONL 파일 로드 시 각 객체의 특정 필드를 메타데이터로 추가하는 함수
def metadata_func(record: dict, metadata: dict) -> dict:
    metadata["sender"] = record.get("sender")
    metadata["timestamp"] = record.get("timestamp")
    return metadata

jsonl_loader_with_meta = JSONLoader(
    file_path="../data/kakao_chat.jsonl", # 데이터 경로 수정
    jq_schema=".",                 # 각 JSON 객체 전체를 가져옴
    content_key="content",         # 'content' 필드를 page_content로 사용
    metadata_func=metadata_func,   # 메타데이터 추출 함수 적용
    json_lines=True,               # JSONL 형식 (한 줄에 하나의 JSON 객체)
)

jsonl_docs_with_meta = jsonl_loader_with_meta.load()
json_docs = jsonl_docs_with_meta 

In [5]:
# 문장을 구분하여 분할 (마침표, 느낌표, 물음표 다음에 공백이 오는 경우 문장의 끝으로 판단)
from langchain_text_splitters import CharacterTextSplitter

text_splitter_regex = CharacterTextSplitter(
    chunk_size=20, # 정규식으로 분리된 조각들을 합칠 때의 최대 크기.
                   # 실제 청크는 이보다 훨씬 작을 수 있음 (정규식 분할 우선).
    chunk_overlap=0, # 문장 단위 분할이므로 중복을 0으로 설정 (일반적).
    separator=r'(?<=[.!?])\s+', # 문장 끝 구두점 뒤 공백을 기준으로 분할하는 정규표현식
                                # 구두점은 유지되고, 뒤따르는 공백이 separator로 사용됨.
    is_separator_regex=True,
    keep_separator=False, # 구분자(여기서는 `\s+`에 해당하는 공백)를 청크에 포함하지 않음.
                         # True로 하면 다음 청크 시작 부분에 공백이 추가될 수 있음.
                         # False로 하여 문장 뒤 공백을 제거하는 효과.
)

regex_texts = text_splitter_regex.split_documents(json_docs) 
print(f"생성된 텍스트 청크 수: {len(regex_texts)}")
chunk_lengths_regex = [len(text.page_content) for text in regex_texts]
print(f"각 청크의 길이 (처음 10개): {chunk_lengths_regex[:10]}")
print()

생성된 텍스트 청크 수: 9
각 청크의 길이 (처음 10개): [31, 9, 15, 7, 11, 11, 27, 13, 13]



In [6]:
# 각 청크의 내용 확인 (문장 단위로 잘 분할되었는지)
for i, text_chunk in enumerate(regex_texts[:5]): # 처음 5개 청크만 확인
    print(f"--- 정규식 청크 {i+1} (길이: {len(text_chunk.page_content)}) ---")
    print(text_chunk.page_content)
    print("=" * 70 + "\n")

--- 정규식 청크 1 (길이: 31) ---
안녕하세요 여러분, 오늘 회의 시간 확인차 연락드립니다.

--- 정규식 청크 2 (길이: 9) ---
네, 안녕하세요.

--- 정규식 청크 3 (길이: 15) ---
오후 2시에 하기로 했어요.

--- 정규식 청크 4 (길이: 7) ---
확인했습니다.

--- 정규식 청크 5 (길이: 11) ---
회의실은 어디인가요?



### 1-3. 토큰 수를 기반으로 분할

- LLM은 내부적으로 텍스트를 토큰(token) 단위로 처리함.
- 글자 수보다는 토큰 수를 기준으로 텍스트를 분할하는 것이 LLM의 컨텍스트 윈도우를 보다 정확하게 관리하는 방법임.
- LangChain은 다양한 토크나이저(tiktoken, Hugging Face tokenizers 등)와 연동하여 토큰 기반 분할을 지원함.

**장점:**
- LLM의 실제 처리 단위인 토큰을 기준으로 분할하므로 컨텍스트 윈도우 관리가 용이함.
- 모델별 토크나이저를 사용하여 해당 모델에 최적화된 분할이 가능함.
- 글자 수 기반 분할보다 LLM의 실제 입력 제한에 더 가깝게 청크 크기를 제어할 수 있음.

**단점:**
- 어떤 토크나이저를 사용하느냐에 따라 분할 결과와 토큰 수가 달라짐.
- 토큰화 과정 자체에 약간의 연산 비용이 추가됨 (일반적으로는 미미함).
- 토큰 경계와 의미론적 경계가 항상 일치하지는 않을 수 있음.

**💡 팁 및 노하우:**
- **모델 호환성**: 사용하려는 LLM 또는 임베딩 모델과 동일하거나 호환되는 토크나이저를 사용하는 것이 가장 정확하고 효과적임.
- **토큰 vs 글자 수**: 토큰 수는 글자 수와 직접적인 비례 관계가 아님. 특히 한국어와 같이 복잡한 언어나 특수 문자가 많은 경우, 글자 수 기반 예측보다 실제 토큰 수를 확인하는 것이 중요함.
- `RecursiveCharacterTextSplitter.from_tiktoken_encoder()` 또는 `from_huggingface_tokenizer()`와 같은 헬퍼 함수를 사용하면 편리하게 토큰 기반 분할기를 생성할 수 있음.

#### (1) `tiktoken` 사용
- OpenAI에서 만든 BPE (Byte Pair Encoding) 기반 토크나이저 라이브러리임.
- GPT 시리즈 (예: gpt-3.5-turbo, gpt-4, text-embedding-ada-002 등) 모델들이 사용하는 토큰화 방식을 따름.
- `RecursiveCharacterTextSplitter.from_tiktoken_encoder()` 메서드를 사용하면 특정 OpenAI 모델의 토크나이저를 기준으로 분할할 수 있음.
  - `encoding_name`: `cl100k_base` (대부분의 최신 OpenAI 모델), `p50k_base` 등 Tiktoken 인코딩 이름을 직접 지정할 수 있음.
  - `model_name`: `gpt-4o-mini`, `text-embedding-3-small` 등 OpenAI 모델 이름을 지정하면 해당 모델의 기본 인코딩을 사용함.

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Tiktoken을 사용하여 토큰 수 기준으로 분할하는 TextSplitter 생성
text_splitter_tiktoken = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    # encoding_name="cl100k_base",  # text-embedding-ada-002, gpt-3.5-turbo, gpt-4 등 최신 모델용 인코딩
    model_name="gpt-4o-mini", # 모델 이름을 지정하여 해당 모델의 토크나이저 사용 (더 권장)
    chunk_size=300,  # 각 청크의 최대 토큰 수
    chunk_overlap=50, # 청크 간 중복되는 토큰 수
    # separators 등 RecursiveCharacterTextSplitter의 다른 파라미터도 사용 가능 (예: ["\n\n", "\n", ". ", " ", ""])
    # separators를 명시하지 않으면 tiktoken 라이브러리가 내부적으로 사용하는 기본 구분자 혹은 글자 단위로 분할 시도
)

# PDF 문서의 첫 페이지만 분할 (pdf_docs[:1]) - 예시를 위해 일부만 사용
chunks_tiktoken = text_splitter_tiktoken.split_documents(pdf_docs[:1])

print(f"생성된 청크 수: {len(chunks_tiktoken)}")
chunk_char_lengths_tiktoken = [len(chunk.page_content) for chunk in chunks_tiktoken]
print(f"각 청크의 글자 수: {chunk_char_lengths_tiktoken}")
print("\n--- 각 청크 미리보기 (Tiktoken) ---")
# 각 청크의 시작 부분과 끝 부분 확인
for i, chunk in enumerate(chunks_tiktoken[:3]): # 처음 3개 청크만 확인
    print(f"\n--- 청크 {i+1} (글자 수: {len(chunk.page_content)}) ---")
    print("[시작 부분]")
    print(chunk.page_content[:100])
    print("\n[...중략...]\n")
    print("[끝 부분]")
    print(chunk.page_content[-100:])
    print("=" * 70)

생성된 청크 수: 3
각 청크의 글자 수: [1140, 1389, 783]

--- 각 청크 미리보기 (Tiktoken) ---

--- 청크 1 (글자 수: 1140) ---
[시작 부분]
Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and

[...중략...]

[끝 부분]
w these models to
be superior in quality while being more parallelizable and requiring significantly

--- 청크 2 (글자 수: 1389) ---
[시작 부분]
based solely on attention mechanisms, dispensing with recurrence and convolutions
entirely. Experime

[...중략...]

[끝 부분]
iki designed, implemented, tuned and evaluated countless model variants in our original codebase and

--- 청크 3 (글자 수: 783) ---
[시작 부분]
attention and the parameter-free position representation and became the other person involved in nea

[...중략...]

[끝 부분]
ormation Processing Systems (NIPS 2017), Long Beach, CA, USA.arXiv:1706.03762v7  [cs.CL]  2 Aug 2023


**Tiktoken으로 실제 토큰 수 확인**

분할된 각 청크가 실제로 목표한 토큰 수 (`chunk_size=300`)에 근접하는지, 그리고 `chunk_overlap`이 어떻게 적용되었는지 확인해봄.

In [8]:
import tiktoken

# text_splitter_tiktoken에서 사용한 것과 동일한 인코딩/모델을 지정해야 정확함.
# tokenizer = tiktoken.get_encoding("cl100k_base") # encoding_name 사용 시
tokenizer_gpt4omini = tiktoken.encoding_for_model("gpt-4o-mini") # model_name 사용 시

print("--- 각 청크의 실제 토큰 수 (Tiktoken gpt-4o-mini) ---")
for i, chunk in enumerate(chunks_tiktoken[:5]): # 처음 5개 청크 확인
    tokens = tokenizer_gpt4omini.encode(chunk.page_content)
    print(f"청크 {i+1}: {len(tokens)} 토큰")
    # print(f"  첫 10개 토큰 ID: {tokens[:10]}")
    # token_strings = [tokenizer_gpt4omini.decode([token]) for token in tokens[:10]]
    # print(f"  첫 10개 토큰 문자열: {token_strings}")
    # print("-" * 50)

--- 각 청크의 실제 토큰 수 (Tiktoken gpt-4o-mini) ---
청크 1: 275 토큰
청크 2: 287 토큰
청크 3: 164 토큰


#### (2) Hugging Face 토크나이저 사용
- Hugging Face `transformers` 라이브러리에서 제공하는 다양한 오픈소스 모델의 토크나이저를 사용할 수 있음.
- `RecursiveCharacterTextSplitter.from_huggingface_tokenizer()` 메서드를 사용함.
  - `tokenizer`: Hugging Face `transformers.PreTrainedTokenizerBase`의 인스턴스를 직접 전달함.
- **장점**: OpenAI 모델 외에 다양한 오픈소스 LLM(예: Llama, Mistral, 한국어 특화 모델 등)에 맞는 토큰화 방식을 적용할 수 있음.
- **단점**: 해당 토크나이저를 미리 로드해야 하며, 모델에 따라 토큰화 방식이 매우 다양할 수 있음.

In [9]:
from transformers import AutoTokenizer

# 예시: BAAI/bge-m3 모델의 토크나이저 로드 (주로 임베딩 모델로 사용됨)
# 실제 사용할 LLM이나 임베딩 모델에 맞는 토크나이저를 선택해야 함.
hf_tokenizer_bge_m3 = AutoTokenizer.from_pretrained("BAAI/bge-m3")
print(hf_tokenizer_bge_m3)

XLMRobertaTokenizerFast(name_or_path='BAAI/bge-m3', vocab_size=250002, model_max_length=8192, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True, added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	250001: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=False, special=True),
}
)


In [10]:
# Hugging Face 토크나이저 인코딩 테스트 (한국어 예시)
sample_text_ko = "안녕하세요. 반갑습니다."
tokens_hf_bge_m3 = hf_tokenizer_bge_m3.encode(sample_text_ko)
print(f"'{sample_text_ko}' -> 토큰 ID: {tokens_hf_bge_m3}")

'안녕하세요. 반갑습니다.' -> 토큰 ID: [0, 107687, 5, 20451, 54272, 16367, 5, 2]


In [11]:
# 토큰 ID를 실제 토큰 문자열로 변환하여 확인
print(f"토큰 문자열: {hf_tokenizer_bge_m3.convert_ids_to_tokens(tokens_hf_bge_m3)}")

토큰 문자열: ['<s>', '▁안녕하세요', '.', '▁반', '갑', '습니다', '.', '</s>']


In [12]:
# 토큰 ID를 다시 원본 텍스트와 유사한 형태로 디코딩
print(f"디코딩된 텍스트: {hf_tokenizer_bge_m3.decode(tokens_hf_bge_m3)}")

디코딩된 텍스트: <s> 안녕하세요. 반갑습니다.</s>


In [13]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Hugging Face 토크나이저를 사용하여 토큰 수 기준으로 분할하는 TextSplitter 생성
text_splitter_hf = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=hf_tokenizer_bge_m3, # 미리 로드한 Hugging Face 토크나이저 전달
    chunk_size=300, 
    chunk_overlap=50,
    # separators 등 RecursiveCharacterTextSplitter의 다른 파라미터도 사용 가능
)

# PDF 문서의 첫 페이지만 분할 (pdf_docs[:1]) - 예시를 위해 일부만 사용
chunks_hf = text_splitter_hf.split_documents(pdf_docs[:1])

In [14]:
print(f"생성된 청크 수: {len(chunks_hf)}")
chunk_char_lengths_hf = [len(chunk.page_content) for chunk in chunks_hf]
print(f"각 청크의 글자 수: {chunk_char_lengths_hf}")
print()

print("--- 각 청크의 실제 토큰 수 (HuggingFace BAAI/bge-m3) ---")
for i, chunk in enumerate(chunks_hf[:5]): # 처음 5개 청크 확인
    # .encode()는 토큰 ID 리스트를 반환함. 길이를 통해 토큰 수 확인.
    tokens = hf_tokenizer_bge_m3.encode(chunk.page_content)
    print(f"청크 {i+1}: {len(tokens)} 토큰")
    # print(f"  첫 10개 토큰 ID: {tokens[:10]}")
    # token_strings = hf_tokenizer_bge_m3.convert_ids_to_tokens(tokens[:10]) 
    # print(f"  첫 10개 토큰 문자열: {token_strings}")
    # print("=" * 70)

생성된 청크 수: 3
각 청크의 글자 수: [1214, 1307, 783]

--- 각 청크의 실제 토큰 수 (HuggingFace BAAI/bge-m3) ---
청크 1: 301 토큰
청크 2: 293 토큰
청크 3: 181 토큰


### 1-4. Semantic Chunking (시맨틱 청킹)

- `SemanticChunker`는 고정된 크기나 규칙 기반이 아닌, 문장 간의 의미론적 유사성을 기반으로 텍스트를 분할함.
- 임베딩 모델을 사용하여 각 문장의 임베딩 벡터를 계산하고, 인접한 문장들 간의 코사인 유사도를 측정함. 이 유사도가 특정 임계값(breakpoint)을 기준으로 크게 변하는 지점에서 청크를 나눔. 즉, 의미적으로 관련성이 높은 문장들을 하나의 청크로 묶으려는 시도임.

**주요 파라미터:**
- `embeddings`: 문장의 의미론적 유사도를 계산하는 데 사용할 임베딩 모델 (예: `OpenAIEmbeddings`, `HuggingFaceEmbeddings`).
- `breakpoint_threshold_type`: 유사도 변화의 기준점을 정하는 방식.
  - `"percentile"` (기본값): 유사도 분포의 특정 백분위수를 기준점으로 사용함.
  - `"standard_deviation"`: 평균에서 표준편차의 특정 배수만큼 떨어진 지점을 기준점으로 사용함.
  - `"gradient"`: 유사도 값의 변화율(기울기)이 급격히 변하는 지점을 찾으며, 문맥 전환을 더 잘 감지할 수 있음.
  - `"interquartile"`: 사분위수 범위를 사용하여 기준점을 설정함.
- `breakpoint_threshold_amount` (또는 `percentile_threshold`, `threshold` 등 타입에 따라 다름): 기준점 타입에 따른 구체적인 값.

**장점:**
- 의미론적으로 응집력 있는 청크를 생성하여 RAG의 검색 품질 및 답변 생성 품질을 향상시킬 잠재력이 있음.
- 고정 크기 분할보다 문맥 유지가 잘 될 수 있으며, 의미적으로 연관된 내용을 하나의 청크로 묶어줌.
- 문맥이 갑자기 바뀌는 부분을 기준으로 분할하므로, 정보 손실을 최소화할 수 있음.

**단점:**
- **실험적 기능**: LangChain 내에서 아직 실험적인 기능으로 분류될 수 있으며, API나 동작 방식이 변경될 수 있음. (현재는 `langchain_experimental`에 위치)
- **계산 비용**: 모든 문장에 대해 임베딩을 계산하고 유사도를 비교해야 하므로, 다른 분할 방식보다 연산량이 많고 시간이 오래 걸릴 수 있음.
- 임베딩 모델의 품질과 데이터 특성에 따라 분할 결과가 크게 달라질 수 있음.
- 최적의 `breakpoint_threshold_type`과 관련 값을 찾기 위한 실험이 필요하며, 데이터에 민감할 수 있음.
- 청크의 크기가 매우 가변적일 수 있음.

**💡 팁 및 노하우:**
- **임베딩 모델 선택**: 문서의 내용과 언어에 적합한 고품질 임베딩 모델을 사용하는 것이 중요함. 한국어 문서라면 한국어 특화 임베딩 모델 사용을 고려해야 함.
- **임계값 튜닝**: `breakpoint_threshold_type`과 관련 값은 데이터셋에 따라 민감하게 반응할 수 있으므로, 여러 옵션을 테스트해보는 것이 좋음. 'gradient' 방식이 의미 변화에 민감하게 반응하는 경향이 있어 시작점으로 고려해볼 만함.
- **사전 문장 분리**: `SemanticChunker`는 내부적으로 텍스트를 문장 단위로 먼저 나눔. 따라서 입력 텍스트에 대한 적절한 문장 분리(sentence splitting)가 선행되거나, `SemanticChunker`가 사용하는 문장 분리 로직을 이해하는 것이 중요함.
- **활용 분야**: 의미론적 일관성이 매우 중요한 고급 RAG 시스템이나, 특정 도메인 문서(예: 법률 문서, 연구 논문)의 심층 분석에 유용할 수 있음. 다만, 계산 비용과 실험적 성격을 고려해야 함.

In [15]:
from dotenv import load_dotenv
load_dotenv() 

True

In [16]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

# SemanticChunker는 임베딩 모델을 사용하여 문장 간 유사도를 계산함
# OpenAIEmbeddings 외에도 HuggingFaceEmbeddings 등 사용 가능
text_splitter_semantic = SemanticChunker(
    embeddings=OpenAIEmbeddings(model="text-embedding-3-small"), # OpenAI 임베딩 사용 (API 키 필요)
    breakpoint_threshold_type="gradient",  # 기준점 타입: 기울기 변화 감지 (문맥 변화에 민감)
    # breakpoint_threshold_type="percentile", percentile_threshold=95, # 예: 상위 5% 변화 지점
    # breakpoint_threshold_type="standard_deviation", breakpoint_threshold_amount=2, # 예: 표준편차 2배 이상 변화
    # add_start_index=True # 메타데이터에 청크 시작 인덱스 추가 여부 (토큰 기반으로 문서를 재조합할 때 유용)
    )


In [17]:
# PDF 문서의 첫 페이지만 분할 (pdf_docs[:1]) - 예시를 위해 일부만 사용
# 주의: SemanticChunker는 내부적으로 문장 단위로 나누고 임베딩을 계산하므로, 
# 입력 문서가 클 경우 상당한 시간이 소요될 수 있습니다.
chunks_semantic = text_splitter_semantic.split_documents(pdf_docs[:1])

print(f"생성된 청크 수 (SemanticChunker): {len(chunks_semantic)}")
chunk_char_lengths_semantic = [len(chunk.page_content) for chunk in chunks_semantic]
print(f"각 청크의 글자 수: {chunk_char_lengths_semantic}")
print()

# Tiktoken 토크나이저로 각 시맨틱 청크의 토큰 수 확인 (참고용)
# SemanticChunker가 사용한 임베딩 모델의 토크나이저와 다를 수 있으므로, 단순 참고용임.
tokenizer_for_semantic_check = tiktoken.encoding_for_model("gpt-4o-mini")

print("--- 각 시맨틱 청크 미리보기 및 토큰 수 (참고용) ---")
for i, chunk in enumerate(chunks_semantic[:5]): # 처음 5개 청크 확인
    tokens = tokenizer_for_semantic_check.encode(chunk.page_content)
    print(f"\n--- 시맨틱 청크 {i+1} (글자 수: {len(chunk.page_content)}, 토큰 수: {len(tokens)}) ---")
    print(chunk.page_content[:200]) # 내용 일부 출력
    if len(chunk.page_content) > 200:
        print("[...]")
    print("=" * 70)

생성된 청크 수 (SemanticChunker): 2
각 청크의 글자 수: [1736, 1116]

--- 각 시맨틱 청크 미리보기 및 토큰 수 (참고용) ---

--- 시맨틱 청크 1 (글자 수: 1736, 토큰 수: 415) ---
Provided proper attribution is provided, Google hereby grants permission to
reproduce the tables and figures in this paper solely for use in journalistic or
scholarly works. Attention Is All You Need

[...]

--- 시맨틱 청크 2 (글자 수: 1116, 토큰 수: 235) ---
∗Equal contribution. Listing order is random. Jakob proposed replacing RNNs with self-attention and started
the effort to evaluate this idea. Ashish, with Illia, designed and implemented the first Tra
[...]
